In [125]:
#导入需要使用的库
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import os

In [126]:
train_data= pd.read_csv(r'./data/used_car_train_20200313.csv', sep=' ')
test_data= pd.read_csv(r'./data/used_car_testB_20200421.csv', sep=' ')

In [127]:
# 合并方便后面的操作
df = pd.concat([train_data, test_data], ignore_index=True)

In [128]:
#选择需要使用的特征标签，由于nn会生成大量的特征，我们只需要保留原始特征和刻画几个明显特征即可
feature = ['model','brand','bodyType','fuelType','kilometer','notRepairedDamage','power','regDate_month','creatDate_year','creatDate_month'
    ,'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6',
       'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14','car_age_day','car_age_year','regDate_year','name_count']

In [129]:
#处理异常数据
df.drop(df[df['seller'] == 1].index, inplace=True)
#记录一下df的price
df_copy = df

In [130]:
# power的范围
df['power'][df['power']>600]=600

In [131]:
df['notRepairedDamage'].value_counts()

0.0    148585
-       32392
1.0     19022
Name: notRepairedDamage, dtype: int64

In [132]:
#notRepairedDamage的值是0和1，然后为-的值设置为0.5，在将它进行标签转换，0->1;0.5->2;1->3;这样符合神经网络的特征提取，不确定值位于两个确定值的中间～
df.replace(to_replace = '-', value = 0.5, inplace = True)
df['notRepairedDamage'].value_counts()

0.0    148585
0.5     32392
1.0     19022
Name: notRepairedDamage, dtype: int64

In [133]:
le = LabelEncoder()
df['notRepairedDamage'] = le.fit_transform(df['notRepairedDamage'].astype(str))

In [134]:
df['notRepairedDamage'].value_counts()

0    148585
1     32392
2     19022
Name: notRepairedDamage, dtype: int64

In [135]:
#日期处理
from datetime import datetime
def date_process(x):
    year = int(str(x)[:4])
    month = int(str(x)[4:6])
    day = int(str(x)[6:8])

    if month < 1:
        month = 7

    date = datetime(year, month, day)
    return date

In [136]:
df['regDates'] = df['regDate'].apply(date_process)
df['creatDates'] = df['creatDate'].apply(date_process)
df['regDate_year'] = df['regDates'].dt.year
df['regDate_month'] = df['regDates'].dt.month
df['regDate_day'] = df['regDates'].dt.day
df['creatDate_year'] = df['creatDates'].dt.year
df['creatDate_month'] = df['creatDates'].dt.month
df['creatDate_day'] = df['creatDates'].dt.day
df['car_age_day'] = (df['creatDates'] - df['regDates']).dt.days
df['car_age_year'] = round(df['car_age_day'] / 365, 1)

In [137]:
#对name进行挖掘，对name进行分组。并且对SaleID计数
df['name_count'] = df.groupby(['name'])['SaleID'].transform('count')
df['name_count']

0         108
1          29
2           3
3           2
4           1
         ... 
199995      1
199996      1
199997      1
199998      1
199999      1
Name: name_count, Length: 199999, dtype: int64

In [138]:
df.drop(['creatDates','regDates'],axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199999 entries, 0 to 199999
Data columns (total 40 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   SaleID             199999 non-null  int64  
 1   name               199999 non-null  int64  
 2   regDate            199999 non-null  int64  
 3   model              199998 non-null  float64
 4   brand              199999 non-null  int64  
 5   bodyType           193989 non-null  float64
 6   fuelType           188395 non-null  float64
 7   gearbox            192050 non-null  float64
 8   power              199999 non-null  int64  
 9   kilometer          199999 non-null  float64
 10  notRepairedDamage  199999 non-null  int32  
 11  regionCode         199999 non-null  int64  
 12  seller             199999 non-null  int64  
 13  offerType          199999 non-null  int64  
 14  creatDate          199999 non-null  int64  
 15  price              149999 non-null  float64
 16  v_

In [139]:
col = list(df.columns)
print(type(col))
print(col)

<class 'list'>
['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode', 'seller', 'offerType', 'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14', 'regDate_year', 'regDate_month', 'regDate_day', 'creatDate_year', 'creatDate_month', 'creatDate_day', 'car_age_day', 'car_age_year', 'name_count']


In [140]:
#填充众数
df.fillna(df.median(),inplace= True)

In [141]:
#特征归一化
scaler = MinMaxScaler()
scaler.fit(df[col].values)

MinMaxScaler()

In [142]:
df= scaler.transform(df[col].values)

In [143]:
## 切割数据,导出数据,作为神经网络的训练数据
nn_data = pd.DataFrame(df,columns=col)
nn_data['price']=np.array(df_copy['price'])
nn_data['SaleID']=np.array(df_copy['SaleID'])
print(nn_data.shape)
print(nn_data)

(199999, 40)
        SaleID      name   regDate     model     brand  bodyType  fuelType  \
0            0  0.003740  0.540610  0.121457  0.153846  0.142857       0.0   
1            1  0.011493  0.498733  0.161943  0.025641  0.285714       0.0   
2            2  0.075575  0.540614  0.465587  0.384615  0.142857       0.0   
3            3  0.365145  0.211048  0.441296  0.256410  0.000000       0.0   
4            4  0.564396  0.871030  0.445344  0.128205  0.142857       0.0   
...        ...       ...       ...       ...       ...       ...       ...   
199994  249995  0.566241  0.543110  0.016194  0.102564  0.000000       0.0   
199995  249996  0.776548  0.913756  0.263158  0.025641  0.000000       0.0   
199996  249997  0.673389  0.543964  0.016194  0.102564  0.000000       0.0   
199997  249998  0.728640  0.458938  0.161943  0.025641  0.571429       0.0   
199998  249999  0.397344  0.749166  0.129555  0.205128  0.142857       0.0   

        gearbox     power  kilometer  ...      v_1

In [144]:
nn_data

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_14,regDate_year,regDate_month,regDate_day,creatDate_year,creatDate_month,creatDate_day,car_age_day,car_age_year,name_count
0,0,0.003740,0.540610,0.121457,0.153846,0.142857,0.0,0.0,0.100000,0.827586,...,0.490716,0.541667,0.272727,0.090909,1.0,0.272727,0.100000,0.472577,0.472222,0.285333
1,1,0.011493,0.498733,0.161943,0.025641,0.285714,0.0,0.0,0.000000,1.000000,...,0.446701,0.500000,0.181818,0.000000,1.0,0.181818,0.266667,0.513139,0.511905,0.074667
2,2,0.075575,0.540614,0.465587,0.384615,0.142857,0.0,0.0,0.271667,0.827586,...,0.415429,0.541667,0.272727,0.181818,1.0,0.272727,0.033333,0.472249,0.472222,0.005333
3,3,0.365145,0.211048,0.441296,0.256410,0.000000,0.0,1.0,0.321667,1.000000,...,0.399071,0.208333,0.727273,0.636364,1.0,0.181818,0.366667,0.771344,0.769841,0.002667
4,4,0.564396,0.871030,0.445344,0.128205,0.142857,0.0,0.0,0.113333,0.310345,...,0.557057,0.875000,0.000000,0.181818,1.0,0.181818,0.400000,0.161378,0.162698,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,249995,0.566241,0.543110,0.016194,0.102564,0.000000,0.0,1.0,0.250000,1.000000,...,0.452033,0.541667,0.818182,0.363636,1.0,0.181818,0.266667,0.449460,0.448413,0.000000
199995,249996,0.776548,0.913756,0.263158,0.025641,0.000000,0.0,0.0,0.298333,0.241379,...,0.467135,0.916667,0.272727,0.727273,1.0,0.181818,0.733333,0.112092,0.115079,0.000000
199996,249997,0.673389,0.543964,0.016194,0.102564,0.000000,0.0,1.0,0.245000,0.827586,...,0.448351,0.541667,1.000000,0.909091,1.0,0.181818,0.500000,0.442918,0.444444,0.000000
199997,249998,0.728640,0.458938,0.161943,0.025641,0.571429,0.0,1.0,0.293333,1.000000,...,0.451361,0.458333,0.545455,0.090909,1.0,0.181818,0.866667,0.541489,0.539683,0.000000


In [145]:
train_num = df.shape[0]-50000
nn_data[0:int(train_num)].to_csv('./data/train_nn.csv', index=0, sep=' ')
nn_data[train_num:train_num+50000].to_csv('./data/test_nn.csv', index=0, sep=' ')